In [1]:
print("hi")

hi


In [30]:
# Set logging for the queries
import logging
import pickle
from langchain.vectorstores.base import VectorStoreRetriever
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [31]:
to_file = "vector_src/ss_src-vectorstore.pkl"
with open(to_file, "rb") as f:
    vectorstore = pickle.load(f)
retriever = VectorStoreRetriever(vectorstore=vectorstore)

In [35]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI
from query_data import get_llm

llm=get_llm('local')
retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

In [36]:
llm.predict("你是谁?你和gpt-3.5的差距有多大")

'我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。\n\nGPT-3.5 是一个由 OpenAI 于 2022 年 11 月推出的语言模型，是 GPT-3 的升级版。GPT-3.5 具有更强的语言理解能力和生成能力，可以用于自动写作、机器翻译、代码调试等领域。\n\n me and gpt-3.5 are both artificial intelligence assistants based on language models. However, the difference between me and gpt-3.5 lies in the model architecture and pre-training target. me is a human-assistant based on the GLM model, while gpt-3.5 is an artificial intelligence assistant based on the GPT model. In other words, gpt-3.5 is more advanced than me in terms of model scale and performance.'

In [37]:
question = "国家的税收策略是什么?"

docs = retriever.get_relevant_documents(query=question)
len(docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. ', '', '   国家的税收策略是什么?', '   ', '   国家税收策略包括哪些方面?', '   ', '   如何制定有效的国家税收策略?', '   ', '2. ', '', '   税收策略对国家的重要性是什么?', '   ', '   国家税收策略的制定过程是什么?', '   ', '   如何评估国家税收策略的有效性?', '   ', '3. ', '', '   国家税收策略的主要目标是什么?', '   ', '   如何平衡国家税收策略与其他政策目标?', '   ', '   在不同国家中,税收策略有什么不同?']
INFO:langchain.retrievers.multi_query:Generated queries: ['What are the limitations of distance-based similarity search? ', '', '1. How does distance-based similarity search fall short in retrieving relevant documents?', '2. What are the drawbacks of relying solely on distance-based similarity search for document retrieval?', '3. In what ways does distance-based similarity search have limitations in retrieving relevant documents?']
INFO:langchain.retrievers.multi_query:Generated queries: ['What are the benefits of exercise for mental health? ', '', '1. How does exercise impact mental well-being?', '2. What positive effects does exercise have on ment

28

In [38]:
docs

[Document(page_content='：：', metadata={'source': 'ss_src/税制结构对我国城乡居民消费水平差距的影响研究_王雨晴.pdf', 'page': 1}),
 Document(page_content='：：', metadata={'source': 'ss_src/税制结构对居民消费的影响研究_霍秋雯.pdf', 'page': 1}),
 Document(page_content='', metadata={'source': 'ss_src/税收凸显性对居民汽车消费行为影响研究_李婉婷.pdf', 'page': 39}),
 Document(page_content="'''''''''", metadata={'source': 'ss_src/个人所得税费用扣除标准提高对城镇居民消费的影响研究_郭志勇.pdf', 'page': 8}),
 Document(page_content="'''''", metadata={'source': 'ss_src/税收负担对居民消费的影响_杨林辉.pdf', 'page': 7}),
 Document(page_content='-', metadata={'source': 'ss_src/个人所得税费用扣除标准提高对城镇居民消费的影响研究_郭志勇.pdf', 'page': 9}),
 Document(page_content='水平的情况下就会增加医疗等方面的支出，提升生活质量。', metadata={'source': 'ss_src/个人所得税改革对居民消费的影响研究_杨志坤.pdf', 'page': 49}),
 Document(page_content='费行为具有显著影响。为了进一步检验上述结果的稳健性，本研究采用替换被解释变量、增加遗漏变量和加入滞后变量三种方式进行稳健性检验。', metadata={'source': 'ss_src/税收凸显性对居民汽车消费行为影响研究_李婉婷.pdf', 'page': 44}),
 Document(page_content='此，国家制定税收政策要充分进行宏观调控，从整体上把握税收量与度的关系，协调好纳税个体与社会整体间的平衡关系，保障税收中性，采取一切有效的措施降低纳税人和社会承受的

In [47]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know,必须使用中文输出!".
    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

In [41]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import SequentialChain
from langchain.chains import TransformChain

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

template = """
Your task is to generate 5 different search queries that aim to
answer the user question from multiple perspectives. 
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.You answer MUST in Chinese output.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template,
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [44]:
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
docs = retriever.get_relevant_documents(
    query=question
)
len(docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 我国税收策略的主要内容是什么？', '2. 国家的税收策略是如何制定的？', '3. 税收策略对国家财政收入有什么影响？', '4. 不同国家的税收策略有何不同？', '5. 税收策略在国际贸易中起什么作用？']


14

In [45]:
from langchain.chains import TransformChain

def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs)
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

In [48]:
retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

rag_chain = SequentialChain(
    chains=[retrieval_chain, qa_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text"]
)

In [49]:
out = rag_chain({"question": question})
out["text"]

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 请问我国税收策略的主要目标是什么？', '2. 税收策略是如何影响国家经济的？', '3. 国家的税收策略是如何制定和实施的？', '4. 税收策略对不同阶层和行业的影响有何不同？', '5. 税收策略在国际比较中是如何看待的？']


'国家的税收策略是宏观调控的重要手段之一，其目的是通过调整税收量与度，协调好纳税个体与社会整体间的平衡关系，保障税收中性，采取一切有效的措施降低纳税人和社会承受的额外负担。在实际经济活动中，作为“理性经济人”的纳税人出于使自身经济利益最大化的目的，会将税收负担减轻作为首要目标。因此，国家的税收策略主要是通过调整税收结构，包括流转税和所得税等，来影响和调节经济活动，促进社会经济的发展，实现对经济的调节作用。'